In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [3]:
os.chdir('..')
print('Changed working directory to parent directory')

with open(os.path.expanduser('~/.huggingface/token')) as f:
    os.environ['HF_TOKEN'] = f.read().strip()
    print(f'Hugging Face token loaded: {os.environ['HF_TOKEN'][:3]}...')

torch.set_grad_enabled(False)
print('Gradient tracking disabled')

if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using device: {device}')

Changed working directory to parent directory
Hugging Face token loaded: hf_...
Gradient tracking disabled
Using device: mps


In [4]:
checkpoint = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

### 2. Download model from Hugging Face

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

In [6]:
model = DistilBertForSequenceClassification.from_pretrained(checkpoint).to(device)

In [7]:
inputs = tokenizer(
  'I am in love with it.', 
  return_tensors='pt', 
  padding='max_length', 
  truncation=True,
  max_length=512
).to(device)

logits = model(**inputs).logits
probs = F.softmax(logits, dim=-1)
predicted_class_id = probs.argmax().item()
confidence_score = probs[0, predicted_class_id].item()
predicted_label = model.config.id2label[predicted_class_id]

print({'label': predicted_label, 'score': confidence_score})

{'label': 'POSITIVE', 'score': 0.9997637867927551}


# Sources

1. [Ground truth - Arena::Indirect Object Identification](https://arena-chapter1-transformer-interp.streamlit.app/[1.4.1]_Indirect_Object_Identification)
2. [Interpretability in the wild: A circuit for indirect object identification in GPT-2 small, by Wang, K, et. al.](https://arxiv.org/pdf/2211.00593)
3. [NOTEBOOK - Exploratory Analysis Demo, by Neel Nanda](https://colab.research.google.com/github/TransformerLensOrg/TransformerLens/blob/main/demos/Exploratory_Analysis_Demo.ipynb#scrollTo=WXktSe0CvBdh)
4. [An analogy for understanding transformers, by Callum McDougall](https://www.lesswrong.com/posts/euam65XjigaCJQkcN/an-analogy-for-understanding-transformers)